In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg2
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
archivo_excel = 'eventos_mina.csv'
df_eventos_mina = pd.read_csv(archivo_excel)#.set_index('ID')
df_eventos_mina.head()

,ID,Fecha,Mina_ID,Volumen_KG,Proporcion_Oro,Proporcion_Aluminio,Proporcion_Zinc,Proporcion_Otros,Oro_KG
0,0,2025-05-01,0,13.620370,37.433045,14.061134,4.303811,44.202010,5.098519
1,1,2025-05-01,2,5.438043,36.302236,11.698926,8.373831,43.625006,1.974131
2,2,2025-05-01,2,6.615838,9.835668,6.865219,10.833004,72.466109,0.650712
3,3,2025-05-01,2,10.385402,31.811953,12.731218,4.203875,51.252954,3.303799
4,4,2025-05-01,0,13.282406,9.758914,2.828766,21.625333,65.786987,1.296219


In [ ]:
archivo_excel = 'inicial_minas.csv'
df_inicial_minas = pd.read_csv(archivo_excel).set_index('ID')
df_inicial_minas

,Nombre,Ubicacion,Representante,Telefono,Entrega Volumen Esperada (toneladas/mes),Entrega Oro Esperada (toneladas/mes),Muestras - Porcentaje Oro,Muestras - Porcentaje Aluminio,Muestras - Porcentaje Zinc,Muestras - Porcentaje Otros,Desviacion Estandar - Componentes (%)
ID,,,,,,,,,,,
0,La Maria,Vegachi,Juan Perez,45485456,1.216216,0.4500,37.0,18.0,5.0,40.0,2.5
1,La Juanita,Amaga,Pedro Ramirez,78914581,1.956000,0.7824,40.0,15.0,3.0,42.0,2.5
2,La Paz,Toledo,Maria Rojas,91545314,1.612286,0.5643,35.0,14.0,3.0,48.0,2.5


In [ ]:
dataset_mina = df_eventos_mina.merge(
    df_inicial_minas,
    left_on='Mina_ID',
    right_on='ID',
    how='left')
# ).set_index(
#     'ID'
# )

dataset_mina.head()

,ID,Fecha,Mina_ID,Volumen_KG,Proporcion_Oro,Proporcion_Aluminio,Proporcion_Zinc,Proporcion_Otros,Oro_KG,Nombre,Ubicacion,Representante,Telefono,Entrega Volumen Esperada (toneladas/mes),Entrega Oro Esperada (toneladas/mes),Muestras - Porcentaje Oro,Muestras - Porcentaje Aluminio,Muestras - Porcentaje Zinc,Muestras - Porcentaje Otros,Desviacion Estandar - Componentes (%)
0,0,2025-05-01,0,13.620370,37.433045,14.061134,4.303811,44.202010,5.098519,La Maria,Vegachi,Juan Perez,45485456,1.216216,0.4500,37.0,18.0,5.0,40.0,2.5
1,1,2025-05-01,2,5.438043,36.302236,11.698926,8.373831,43.625006,1.974131,La Paz,Toledo,Maria Rojas,91545314,1.612286,0.5643,35.0,14.0,3.0,48.0,2.5
2,2,2025-05-01,2,6.615838,9.835668,6.865219,10.833004,72.466109,0.650712,La Paz,Toledo,Maria Rojas,91545314,1.612286,0.5643,35.0,14.0,3.0,48.0,2.5
3,3,2025-05-01,2,10.385402,31.811953,12.731218,4.203875,51.252954,3.303799,La Paz,Toledo,Maria Rojas,91545314,1.612286,0.5643,35.0,14.0,3.0,48.0,2.5
4,4,2025-05-01,0,13.282406,9.758914,2.828766,21.625333,65.786987,1.296219,La Maria,Vegachi,Juan Perez,45485456,1.216216,0.4500,37.0,18.0,5.0,40.0,2.5


In [ ]:
fechas_distintas_por_mina = dataset_mina.groupby('Nombre')['Oro_KG'].sum()
fechas_distintas_por_mina/1000


Nombre
La Juanita    0.737601
La Maria      0.397163
La Paz        0.454739
Name: Oro_KG, dtype: float64

In [ ]:
# Calcular los límites de los rangos para Proporcion_Oro
dataset_mina['Limite_Inferior_Oro'] = dataset_mina['Muestras - Porcentaje Oro'] - dataset_mina['Desviacion Estandar - Componentes (%)']
dataset_mina['Limite_Superior_Oro'] = dataset_mina['Muestras - Porcentaje Oro'] + dataset_mina['Desviacion Estandar - Componentes (%)']

# Calcular los límites de los rangos para Proporcion_Aluminio
dataset_mina['Limite_Inferior_Aluminio'] = dataset_mina['Muestras - Porcentaje Aluminio'] - dataset_mina['Desviacion Estandar - Componentes (%)']
dataset_mina['Limite_Superior_Aluminio'] = dataset_mina['Muestras - Porcentaje Aluminio'] + dataset_mina['Desviacion Estandar - Componentes (%)']

# Calcular los límites de los rangos para Proporcion_Oro
dataset_mina['Limite_Inferior_Zinc'] = dataset_mina['Muestras - Porcentaje Zinc'] - dataset_mina['Desviacion Estandar - Componentes (%)']
dataset_mina['Limite_Superior_Zinc'] = dataset_mina['Muestras - Porcentaje Zinc'] + dataset_mina['Desviacion Estandar - Componentes (%)']

# Calcular los límites de los rangos para Proporcion_Aluminio
dataset_mina['Limite_Inferior_Otros'] = dataset_mina['Muestras - Porcentaje Otros'] - dataset_mina['Desviacion Estandar - Componentes (%)']
dataset_mina['Limite_Superior_Otros'] = dataset_mina['Muestras - Porcentaje Otros'] + dataset_mina['Desviacion Estandar - Componentes (%)']
dataset_mina.head()



,ID,Fecha,Mina_ID,Volumen_KG,Proporcion_Oro,Proporcion_Aluminio,Proporcion_Zinc,Proporcion_Otros,Oro_KG,Nombre,...,Muestras - Porcentaje Otros,Desviacion Estandar - Componentes (%),Limite_Inferior_Oro,Limite_Superior_Oro,Limite_Inferior_Aluminio,Limite_Superior_Aluminio,Limite_Inferior_Zinc,Limite_Superior_Zinc,Limite_Inferior_Otros,Limite_Superior_Otros
0,0,2025-05-01,0,13.620370,37.433045,14.061134,4.303811,44.202010,5.098519,La Maria,...,40.0,2.5,34.5,39.5,15.5,20.5,2.5,7.5,37.5,42.5
1,1,2025-05-01,2,5.438043,36.302236,11.698926,8.373831,43.625006,1.974131,La Paz,...,48.0,2.5,32.5,37.5,11.5,16.5,0.5,5.5,45.5,50.5
2,2,2025-05-01,2,6.615838,9.835668,6.865219,10.833004,72.466109,0.650712,La Paz,...,48.0,2.5,32.5,37.5,11.5,16.5,0.5,5.5,45.5,50.5
3,3,2025-05-01,2,10.385402,31.811953,12.731218,4.203875,51.252954,3.303799,La Paz,...,48.0,2.5,32.5,37.5,11.5,16.5,0.5,5.5,45.5,50.5
4,4,2025-05-01,0,13.282406,9.758914,2.828766,21.625333,65.786987,1.296219,La Maria,...,40.0,2.5,34.5,39.5,15.5,20.5,2.5,7.5,37.5,42.5


In [ ]:
# registros_fraudulentos = dataset_mina[
#     (dataset_mina['Proporcion_Oro'] < dataset_mina['Limite_Inferior_Oro']) |
#     (dataset_mina['Proporcion_Oro'] > dataset_mina['Limite_Superior_Oro']) |
#     (dataset_mina['Proporcion_Aluminio'] < dataset_mina['Limite_Inferior_Aluminio']) |
#     (dataset_mina['Proporcion_Aluminio'] > dataset_mina['Limite_Superior_Aluminio']) |
#     (dataset_mina['Proporcion_Zinc'] < dataset_mina['Limite_Inferior_Zinc']) |
#     (dataset_mina['Proporcion_Zinc'] > dataset_mina['Limite_Superior_Zinc']) |
#     (dataset_mina['Proporcion_Otros'] < dataset_mina['Limite_Inferior_Otros']) |
#     (dataset_mina['Proporcion_Otros'] > dataset_mina['Limite_Superior_Otros'])
# ]

# registros_fraudulentos



dataset_mina['Es_Fraudulento'] = (
    (dataset_mina['Proporcion_Oro'] < dataset_mina['Limite_Inferior_Oro']) |
    (dataset_mina['Proporcion_Oro'] > dataset_mina['Limite_Superior_Oro']) |
    (dataset_mina['Proporcion_Aluminio'] < dataset_mina['Limite_Inferior_Aluminio']) |
    (dataset_mina['Proporcion_Aluminio'] > dataset_mina['Limite_Superior_Aluminio']) |
    (dataset_mina['Proporcion_Zinc'] < dataset_mina['Limite_Inferior_Zinc']) |
    (dataset_mina['Proporcion_Zinc'] > dataset_mina['Limite_Superior_Zinc']) |
    (dataset_mina['Proporcion_Otros'] < dataset_mina['Limite_Inferior_Otros']) |
    (dataset_mina['Proporcion_Otros'] > dataset_mina['Limite_Superior_Otros'])
)

resumen_por_mina = dataset_mina.groupby('Nombre').agg(
    Total_Registros=('Nombre', 'size'),
    Registros_Fraudulentos=('Es_Fraudulento', 'sum'),
    Registros_Cumplen=('Es_Fraudulento', lambda x: len(x) - sum(x))
).reset_index()

In [ ]:
resumen_por_mina

,Nombre,Total_Registros,Registros_Fraudulentos,Registros_Cumplen
0,La Juanita,124,95,29
1,La Maria,111,89,22
2,La Paz,228,193,35


In [ ]:
resumen_por_mina['Porcentaje_Fraudulentos'] = (resumen_por_mina['Registros_Fraudulentos'] / resumen_por_mina['Total_Registros']) * 100
resumen_por_mina['Porcentaje_Cumplen'] = (resumen_por_mina['Registros_Cumplen'] / resumen_por_mina['Total_Registros']) * 100
resumen_por_mina

,Nombre,Total_Registros,Registros_Fraudulentos,Registros_Cumplen,Porcentaje_Fraudulentos,Porcentaje_Cumplen
0,La Juanita,124,95,29,76.612903,23.387097
1,La Maria,111,89,22,80.180180,19.819820
2,La Paz,228,193,35,84.649123,15.350877


In [ ]:
datos_mina_1 = dataset_mina[dataset_mina['Nombre'] =='La Maria']




In [ ]:
archivo_excel = 'inicial_joyeros.csv'
df_inicial_joyeros = pd.read_csv(archivo_excel)#.set_index('ID')
df_inicial_joyeros.head()

,ID,Nombre,Telefono,Direccion,NIT
0,0,Daniel Medina,8428646,"Transversal 9 # 29 - 589, Sabaneta",27165040
1,1,Gabriela Vasquez,8425161,"Carrera 26 # 73 - 3, Sabaneta",27297956
2,2,Isabella Rivera,8424810,"Transversal 99 # 40 - 448, Medellin",28946934
3,3,Camila Sanchez,8423481,"Transversal 85 # 5 - 471, Medellin",26027856
4,4,Elena Hernandez,8422048,"Diagonal 32 # 59 - 25, Medellin",28880289


In [ ]:
archivo_excel = 'eventos_joyero.csv'
df_eventos_joyeros = pd.read_csv(archivo_excel)#.set_index('ID')
df_eventos_joyeros.head()

,ID,Fecha,Joyero_ID,Objeto,Volumen_GR,Oro_GR,Kilates,Segunda_Procedencia_Nombre,Segunda_Procedencia_Telefono,Segunda_Procedencia_Cedula,Segunda_Procedencia_Antecedentes_Judiciales
0,0,2025-05-01,8,Collar,55.199564,19.924510,8.662899,Camila Garcia,31122243.0,NaN,NaN
1,1,2025-05-01,2,Collar,62.552246,51.495271,19.757668,Maria Castro,NaN,10821202.0,0.0
2,2,2025-05-01,2,Dije,48.291064,20.186193,10.032262,Gabriela Mendoza,31169855.0,NaN,NaN
3,3,2025-05-01,2,Pulsera,54.359008,36.765484,16.232298,Sara Ruiz,31175374.0,10557903.0,0.0
4,4,2025-05-01,6,Dije,55.479545,23.042313,9.967917,Juliana Aguilar,31084716.0,10524688.0,0.0


In [ ]:

conteo_por_procedencia = df_eventos_joyeros.groupby(['Segunda_Procedencia_Nombre']).size().reset_index(name='conteo')

# # Ordenar por conteo en orden descendente
conteo_ordenado = conteo_por_procedencia.sort_values(by='conteo', ascending=False)
conteo_ordenado

,Segunda_Procedencia_Nombre,conteo
23,Carlos Duarte,10
75,Maria Rodriguez,10
69,Luis Ortiz,3
90,Ricardo Morales,2
91,Sara Arias,2
...,...,...
34,Cristian Ortiz,1
33,Cristian Hernandez,1
32,Cristian Arias,1
31,Carolina Silva,1


In [ ]:
df_eventos_joyeros.groupby('Segunda_Procedencia_Nombre')['Oro_GR'].sum().reset_index(name='total_oro').sort_values(by='total_oro', ascending=False)




,Segunda_Procedencia_Nombre,total_oro
75,Maria Rodriguez,479.787220
69,Luis Ortiz,121.273343
23,Carlos Duarte,86.031215
29,Carolina Castro,81.741980
41,Daniela Torres,59.069901
...,...,...
93,Sara Gutierrez,10.876714
8,Alberto Vasquez,10.133499
102,Valentina Rojas,9.653341
2,Adrian Diaz,9.433570


In [ ]:
df_eventos_joyeros.groupby('Segunda_Procedencia_Nombre').agg(
    Total_Registros=('Segunda_Procedencia_Nombre', 'size'),  # Conteo de registros
    Suma_Oro_GR=('Oro_GR', 'sum')  # Suma de la columna Oro_GR
).reset_index().sort_values(by='Suma_Oro_GR', ascending=False)

,Segunda_Procedencia_Nombre,Total_Registros,Suma_Oro_GR
75,Maria Rodriguez,10,479.787220
69,Luis Ortiz,3,121.273343
23,Carlos Duarte,10,86.031215
29,Carolina Castro,2,81.741980
41,Daniela Torres,1,59.069901
...,...,...,...
93,Sara Gutierrez,1,10.876714
8,Alberto Vasquez,1,10.133499
102,Valentina Rojas,1,9.653341
2,Adrian Diaz,1,9.433570


In [ ]:
df_eventos_joyeros['Es_Sospechoso'] = (df_eventos_joyeros['Segunda_Procedencia_Nombre'].isnull()) | \
                                      (df_eventos_joyeros['Segunda_Procedencia_Telefono'].isnull()) | \
                                      (df_eventos_joyeros['Segunda_Procedencia_Cedula'].isnull())

df_eventos_joyeros.head(5)

,ID,Fecha,Joyero_ID,Objeto,Volumen_GR,Oro_GR,Kilates,Segunda_Procedencia_Nombre,Segunda_Procedencia_Telefono,Segunda_Procedencia_Cedula,Segunda_Procedencia_Antecedentes_Judiciales,Es_Sospechoso
0,0,2025-05-01,8,Collar,55.199564,19.924510,8.662899,Camila Garcia,31122243.0,NaN,NaN,True
1,1,2025-05-01,2,Collar,62.552246,51.495271,19.757668,Maria Castro,NaN,10821202.0,0.0,True
2,2,2025-05-01,2,Dije,48.291064,20.186193,10.032262,Gabriela Mendoza,31169855.0,NaN,NaN,True
3,3,2025-05-01,2,Pulsera,54.359008,36.765484,16.232298,Sara Ruiz,31175374.0,10557903.0,0.0,False
4,4,2025-05-01,6,Dije,55.479545,23.042313,9.967917,Juliana Aguilar,31084716.0,10524688.0,0.0,False


In [ ]:

conteo_sospechosos = df_eventos_joyeros.groupby('Joyero_ID')['Es_Sospechoso'].sum()

# Calcular el total de eventos por Joyero_ID
total_eventos = df_eventos_joyeros.groupby('Joyero_ID').size()

# Calcular el porcentaje de eventos sospechosos por Joyero_ID
porcentaje_sospechosos = (conteo_sospechosos / total_eventos) * 100

# Conteo de 'Maria Rodriguez' y 'Carlos Duarte' en Segunda_Procedencia_Nombre por Joyero_ID
conteo_nombres = df_eventos_joyeros['Segunda_Procedencia_Nombre'].isin(['Maria Rodriguez', 'Carlos Duarte']).groupby(df_eventos_joyeros['Joyero_ID']).sum()

# Crear un DataFrame con todos los cálculos
resultados = pd.DataFrame({
    'Total_True_Es_Sospechoso': conteo_sospechosos,
    'Total_Eventos': total_eventos,
    'Porcentaje_Sospechosos': porcentaje_sospechosos,
    'Registros_Antecedentes_Distintos_Cero': df_eventos_joyeros[df_eventos_joyeros['Segunda_Procedencia_Antecedentes_Judiciales'] == 1].groupby('Joyero_ID').size(),
    'Sospecha_Mucha_Venta': conteo_nombres
})

# Ordenar por el porcentaje de sospechosos en orden descendente

# Ordenar por el porcentaje de sospechosos en orden descendente
resultados = resultados.sort_values(by='Joyero_ID', ascending=True)

resultados


,Total_True_Es_Sospechoso,Total_Eventos,Porcentaje_Sospechosos,Registros_Antecedentes_Distintos_Cero,Sospecha_Mucha_Venta
Joyero_ID,,,,,
0,3,15,20.000000,1.0,2
1,2,21,9.523810,NaN,2
2,3,23,13.043478,NaN,2
3,3,13,23.076923,NaN,2
4,1,8,12.500000,1.0,2
5,2,5,40.000000,1.0,2
6,3,20,15.000000,NaN,2
7,0,6,0.000000,1.0,2
8,3,11,27.272727,1.0,2


In [ ]:
condicion_es_sospechoso = df_eventos_joyeros['Es_Sospechoso'] == True
condicion_antecedentes_distintos_cero = df_eventos_joyeros['Segunda_Procedencia_Antecedentes_Judiciales'] == 1
condicion_sospecha_mucha_venta = df_eventos_joyeros['Segunda_Procedencia_Nombre'].isin(['Maria Rodriguez', 'Carlos Duarte'])

# Aplicar filtro combinando las condiciones con operador OR (|)
condiciones = condicion_es_sospechoso | condicion_antecedentes_distintos_cero | condicion_sospecha_mucha_venta

# Filtrar DataFrame con las condiciones
df_filtrado = df_eventos_joyeros[condiciones]

# Sumar Oro_GR por Joyero_ID en el DataFrame filtrado
suma_oro_por_joyero = df_filtrado.groupby('Joyero_ID')['Oro_GR'].sum()
suma_oro_por_joyero

Joyero_ID
0    182.786357
1     93.558774
2    128.090276
3    174.167530
4    120.794717
5    207.208190
6    101.026809
7    102.370490
8    191.527015
9     66.747883
Name: Oro_GR, dtype: float64

In [ ]:
archivo_excel = 'eventos_barequeros.csv'
df_eventos_bar = pd.read_csv(archivo_excel)#.set_index('ID')
df_eventos_bar.head()

,ID,Fecha,Barequero_ID,Cooperativa_ID,Oro_GR
0,0,2025-05-01,16,0.0,0.280794
1,1,2025-05-01,3,1.0,3.114015
2,2,2025-05-01,2,1.0,2.805140
3,3,2025-05-01,15,2.0,2.630429
4,4,2025-05-02,2,1.0,1.091395


In [ ]:
suma_oro_por_barequero = df_eventos_bar.groupby('Barequero_ID')['Oro_GR'].sum()

# Conteo de registros por Barequero_ID
conteo_registros_por_barequero = df_eventos_bar.groupby('Barequero_ID').size()

# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'Suma_Oro_GR': suma_oro_por_barequero,
    'Conteo_Registros': conteo_registros_por_barequero
})

resultados

,Suma_Oro_GR,Conteo_Registros
Barequero_ID,,
0,10.387754,4
1,50.023728,2
2,9.533503,4
3,6.140546,2
4,14.239437,2
5,8.630165,1
6,14.571670,3
7,12.132953,1
8,6.855528,4
